# Load packages and libraries

In [1]:
.libPaths()
assign(".lib.loc", "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library", envir = environment(.libPaths))
.libPaths()
# sessionInfo()


suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(dplyr)) 
suppressMessages(library(ggplot2)) 
suppressMessages(library(Matrix)) 
suppressMessages(library(data.table)) 
suppressMessages(library(ggpubr)) 
suppressMessages(library(ggplot2))
suppressMessages(library(pheatmap))
suppressMessages(library(presto))
suppressMessages(library("qlcMatrix"))
suppressMessages(library("cowplot"))
suppressMessages(library("RColorBrewer"))
suppressMessages(library("plyr"))
suppressMessages(library("forcats"))
suppressMessages(library('ggeasy'))
suppressMessages(library('dplyr'))
suppressMessages(library("svglite"))
suppressMessages(library("ape"))
suppressMessages(library("ggforce"))
suppressMessages(library("tidyr"))
suppressMessages(library("edgeR"))
suppressMessages(library("apeglm"))
suppressMessages(library("DESeq2"))
suppressMessages(library("tibble")) 
suppressMessages(library(future))

library("optparse")
suppressMessages(library("splitstackshape")) 
suppressMessages(library("ggupset"))


[1] "/group/soranzo/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"

[1] "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"

# Read in featureCounts files and transform to matrix structure

In [2]:
setwd("/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/")

list_files<-dir()

list_files_sel<-list_files[grep("_raw_counts.txt$", list_files)]

str(list_files_sel)
cat("\n")

sample_vector<-gsub("_raw_counts.txt$","",list_files_sel)

str(sample_vector)
cat("\n")

df<-as.data.frame(cbind(list_files_sel,sample_vector))
colnames(df)<-c('file','sample')

df$file<-paste("/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/",df$file, sep='')


str(df)
cat("\n")




 chr [1:6] "MCO_01334_raw_counts.txt" "MCO_01335_raw_counts.txt" ...

 chr [1:6] "MCO_01334" "MCO_01335" "MCO_01338" "MCO_01339" "MCO_01342" ...

'data.frame':	6 obs. of  2 variables:
 $ file  : chr  "/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/MCO_01334_raw_counts.txt" "/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/MCO_01335_raw_counts.txt" "/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/MCO_01338_raw_counts.txt" "/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/MCO_01339_raw_counts.txt" ...
 $ sample: chr  "MCO_01334" "MCO_01335" "MCO_01338" "MCO_01339" ...



In [3]:
DEBUG<-0

# Initialize a variable to store the merged count matrix (optional, but good practice)
all_raw_counts <- NULL 


for(i in 1:dim(df)[1]){

    sample_array_sel<-df$sample[i]

    cat("-------------------------->\t")
    cat(sprintf(as.character(sample_array_sel)))
    cat("\n")

    raw_counts_file<-df$file[i]

    raw_counts_df<-read.table(file=raw_counts_file,sep="\t", skip=1, header=TRUE, stringsAsFactors = FALSE)
    colnames(raw_counts_df)[7]<-sample_array_sel

    if(DEBUG == 1){
        
        cat("raw_counts_df_0\n")
        str(raw_counts_df)
        cat("\n")
    }

    indx_sel<-which(colnames(raw_counts_df)%in%c('Geneid',sample_array_sel))


    raw_counts_df_subset<-raw_counts_df[,indx_sel]

    if(DEBUG == 1){
        
        cat("raw_counts_df_subset_0\n")
        str(raw_counts_df_subset)
        cat("\n")
    }

    # 3. Initialize or Merge the count matrix
    if (is.null(all_raw_counts)) {
        # If this is the first sample, initialize the matrix
        all_raw_counts <- raw_counts_df_subset
        } else {
            # For subsequent samples, merge the new count column 
            # using 'Geneid' as the key to align the rows.
            all_raw_counts <- merge(
                x = all_raw_counts, 
                y = raw_counts_df_subset, 
                by.x = "Geneid", 
                by.y = "Geneid",
                all = TRUE # Keeps all Gene IDs, even if a sample has a count of 0 for a gene.
            )
        }#if (is.null(all_raw_counts)

    if(DEBUG == 1){
        
        cat("all_raw_counts_0\n")
        str(all_raw_counts)
        cat("\n")
    }

    
}#i in 1:length(sample_array)


-------------------------->	MCO_01334
-------------------------->	MCO_01335
-------------------------->	MCO_01338
-------------------------->	MCO_01339
-------------------------->	MCO_01342
-------------------------->	MCO_01343


In [4]:
cat("all_raw_counts_0\n")
str(all_raw_counts)
cat("\n")

all_raw_counts_0
'data.frame':	792 obs. of  7 variables:
 $ Geneid   : chr  "ENSG00000199196" "ENSG00000199231" "ENSG00000199282" "ENSG00000199321" ...
 $ MCO_01334: int  0 0 0 0 0 0 0 0 0 0 ...
 $ MCO_01335: int  0 0 0 0 0 0 0 0 0 0 ...
 $ MCO_01338: int  0 0 0 0 0 0 0 0 0 0 ...
 $ MCO_01339: int  0 0 0 0 0 0 0 0 0 0 ...
 $ MCO_01342: int  0 0 0 0 0 0 0 0 0 0 ...
 $ MCO_01343: int  0 0 2 0 0 0 0 0 0 0 ...



## Force all NAs 0

In [5]:
all_raw_counts[is.na(all_raw_counts)] <- 0

## Convert into a DESeq2 count matrix

In [6]:
all_raw_counts_matrix<-as.matrix(all_raw_counts[,-which(colnames(all_raw_counts) == 'Geneid')])

row.names(all_raw_counts_matrix)<-all_raw_counts$Geneid


cat("all_raw_counts_matrix_0\n")
str(all_raw_counts_matrix)
cat("\n")

all_raw_counts_matrix_0
 int [1:792, 1:6] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:792] "ENSG00000199196" "ENSG00000199231" "ENSG00000199282" "ENSG00000199321" ...
  ..$ : chr [1:6] "MCO_01334" "MCO_01335" "MCO_01338" "MCO_01339" ...



## Check some expression values

In [7]:
all_raw_counts_matrix[which(row.names(all_raw_counts_matrix) == 'SNORD42A'),]

MCO_01334 MCO_01335 MCO_01338 MCO_01339 MCO_01342 MCO_01343 
      275       260      1297       251       681       887

## Count matrix

In [8]:
count_matrix<-all_raw_counts_matrix

cat("count_matrix_0\n")
cat(str(count_matrix))
cat("\n")

count_matrix_0
 int [1:792, 1:6] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:792] "ENSG00000199196" "ENSG00000199231" "ENSG00000199282" "ENSG00000199321" ...
  ..$ : chr [1:6] "MCO_01334" "MCO_01335" "MCO_01338" "MCO_01339" ...



### Sample to condition equivalence

In [9]:
colnames(count_matrix)[which(colnames(count_matrix) == 'MCO_01334')]<-'wt_1'
colnames(count_matrix)[which(colnames(count_matrix) == 'MCO_01335')]<-'rs62237617_1'

colnames(count_matrix)[which(colnames(count_matrix) == 'MCO_01338')]<-'wt_2'
colnames(count_matrix)[which(colnames(count_matrix) == 'MCO_01339')]<-'rs62237617_2'

colnames(count_matrix)[which(colnames(count_matrix) == 'MCO_01342')]<-'wt_3'
colnames(count_matrix)[which(colnames(count_matrix) == 'MCO_01343')]<-'rs62237617_3'

cat("count_matrix_1\n")
cat(str(count_matrix))
cat("\n")

count_matrix_1
 int [1:792, 1:6] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:792] "ENSG00000199196" "ENSG00000199231" "ENSG00000199282" "ENSG00000199321" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...



# Create the colData object

In [10]:
samples<-colnames(count_matrix)

cat("samples/n")
cat(str(samples))


samples/n chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" "wt_3" ...


In [11]:
condition<-gsub("_.+$","",samples)

cat("condition/n")
cat(str(condition))


condition/n chr [1:6] "wt" "rs62237617" "wt" "rs62237617" "wt" "rs62237617"


In [12]:
batch<-gsub("^[^_]+_","",samples)

cat("batch/n")
cat(str(batch))


batch/n chr [1:6] "1" "1" "2" "2" "3" "3"


In [13]:
colData<-as.data.frame(cbind(samples,condition,batch))

colnames(colData)<-c('sample','condition','batch')

cat("colData_0/n")
cat(str(colData))


colData_0/n'data.frame':	6 obs. of  3 variables:
 $ sample   : chr  "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
 $ condition: chr  "wt" "rs62237617" "wt" "rs62237617" ...
 $ batch    : chr  "1" "1" "2" "2" ...


## factorize condition

In [14]:
colData$condition<-factor(colData$condition)
colData$condition<-relevel(colData$condition, ref='wt') ### Nothing works with ordered factors


cat("colData_1/n")
cat(str(colData))

colData_1/n'data.frame':	6 obs. of  3 variables:
 $ sample   : chr  "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
 $ condition: Factor w/ 2 levels "wt","rs62237617": 1 2 1 2 1 2
 $ batch    : chr  "1" "1" "2" "2" ...


# DESeq2

## Create the DESeqDataSetFromMatrix object

In [15]:
dds <- DESeqDataSetFromMatrix(countData = count_matrix,
                              colData = colData,
                              design = ~ condition)

In [16]:
cat("dds_0/n")
#cat(str(dds))

dds_0/n

## Filter out snoRNAs with fewer than  total raw counts

In [17]:
keep <- rowSums(counts(dds)) >= 10

filtered_out<-keep[which(keep == "FALSE")]

cat(str(filtered_out))

 Named logi [1:542] FALSE FALSE FALSE FALSE FALSE FALSE ...
 - attr(*, "names")= chr [1:542] "ENSG00000199196" "ENSG00000199231" "ENSG00000199282" "ENSG00000199321" ...


In [18]:
grep("SNORD42",names(filtered_out))

[1] 523

In [19]:
dds <- dds[keep,]

## estimateSizeFactors----------------------------------


In [20]:
dds<-estimateSizeFactors(dds)

## Run the rest

In [21]:
dds <- DESeq(dds)

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## Extract results

In [22]:
possible_contrasts<-colnames(dds@modelMatrix)[-1] # -1 because 1 is the Intercept term

possible_contrasts

[1] "condition_rs62237617_vs_wt"

In [23]:
resultsNames(dds)

[1] "Intercept"                  "condition_rs62237617_vs_wt"

In [24]:
tmp_results<-results(dds, test= "Wald", name=possible_contrasts[1], independentFiltering=FALSE)
        
#### expand LogFC #########################################

tmp_results <- lfcShrink(dds, 
                         coef = possible_contrasts[1],
                         res=tmp_results,
                         type = "apeglm")


cat("tmp_results_0/n")
#cat(str(tmp_results))

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



tmp_results_0/n

In [25]:
 #### obtain data frame #########################################
        
tmp_tb <- as.data.frame(tmp_results %>%
                          data.frame() %>%
                          rownames_to_column(var = "snoRNA") %>%
                          as_tibble() %>%
                          arrange(padj), stringsAsFactors=F)

tmp_tb$contrast<-possible_contrasts[1]

cat("tmp_tb_0")
cat(str(tmp_tb))

tmp_tb_0'data.frame':	250 obs. of  7 variables:
 $ snoRNA        : chr  "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
 $ baseMean      : num  2.72 1.16 47.85 2.36 1.62 ...
 $ log2FoldChange: num  -4.82e-07 -3.19e-07 5.60e-06 7.28e-07 -2.14e-08 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.47762 0.51146 0.00501 0.26559 0.97 ...
 $ padj          : num  0.997 0.997 0.997 0.997 0.997 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...


In [26]:
tmp_tb$minuslog10padj <- -log10(tmp_tb$padj)
      
tmp_tb$abslogfc<-abs(tmp_tb$log2FoldChange)

cat("tmp_tb_0")
cat(str(tmp_tb))

tmp_tb_0'data.frame':	250 obs. of  9 variables:
 $ snoRNA        : chr  "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
 $ baseMean      : num  2.72 1.16 47.85 2.36 1.62 ...
 $ log2FoldChange: num  -4.82e-07 -3.19e-07 5.60e-06 7.28e-07 -2.14e-08 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.47762 0.51146 0.00501 0.26559 0.97 ...
 $ padj          : num  0.997 0.997 0.997 0.997 0.997 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...
 $ minuslog10padj: num  0.00118 0.00118 0.00118 0.00118 0.00118 ...
 $ abslogfc      : num  4.82e-07 3.19e-07 5.60e-06 7.28e-07 2.14e-08 ...


In [27]:
DE_results<-tmp_tb

## Extract normalized counts

In [28]:
nor_counts<-as.matrix(counts(dds, normalized=TRUE))
      
row.names(nor_counts)<-row.names(nor_counts)

cat("nor_counts_0\n")
str(nor_counts)
cat("\n")


nor_counts_0
 num [1:250, 1:6] 0 0 25.1 1 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:250] "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...



# SAVE results

In [29]:
setwd("/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/")

In [30]:
write.table(DE_results, file="DE_results.tsv", sep="\t", quote=F, row.names=F)

write.table(nor_counts, file="nor_counts.tsv", sep="\t", quote=F)


## Explore results

In [31]:


test<-DE_results[which(DE_results$pvalue <= 0.05),]

cat("test_0\n")
str(test)
cat("\n")

test_0
'data.frame':	3 obs. of  9 variables:
 $ snoRNA        : chr  "ENSG00000273885" "SNORA74B" "SNORD82"
 $ baseMean      : num  47.85 5.53 7.8
 $ log2FoldChange: num  5.60e-06 2.19e-06 2.10e-06
 $ lfcSE         : num  0.00144 0.00144 0.00144
 $ pvalue        : num  0.00501 0.02534 0.0401
 $ padj          : num  0.997 0.997 0.997
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt"
 $ minuslog10padj: num  0.00118 0.00118 0.00118
 $ abslogfc      : num  5.60e-06 2.19e-06 2.10e-06



In [32]:
SIG<-DE_results[which(DE_results$minuslog10padj >= 1.3),]

cat("SIG_0\n")
str(SIG)
cat("\n")

SIG_0
'data.frame':	0 obs. of  9 variables:
 $ snoRNA        : chr 
 $ baseMean      : num 
 $ log2FoldChange: num 
 $ lfcSE         : num 
 $ pvalue        : num 
 $ padj          : num 
 $ contrast      : chr 
 $ minuslog10padj: num 
 $ abslogfc      : num 



# Heatmap

In [33]:
dev.off()

null device 
          1

ERROR: Error in dev.set(dev): no active device and default getOption("device") is invalid


## Load libraries

In [34]:
.libPaths()
assign(".lib.loc", "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library", envir = environment(.libPaths))
.libPaths()
# # sessionInfo()


# .libPaths()
# .libPaths(new = c("/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"))
# .libPaths()
# sessionInfo()

library("optparse")
suppressMessages(library(dplyr)) 
suppressMessages(library(ggplot2)) 
suppressMessages(library(Matrix)) 
suppressMessages(library(data.table)) 
suppressMessages(library(ggpubr)) 
suppressMessages(library(ggplot2))
suppressMessages(library(pheatmap))
suppressMessages(library("cowplot"))
suppressMessages(library("RColorBrewer"))
suppressMessages(library("plyr"))
suppressMessages(library("forcats"))
suppressMessages(library('ggeasy'))
suppressMessages(library('dplyr'))
suppressMessages(library("svglite"))
suppressMessages(library("ape"))
suppressMessages(library("ggforce"))
suppressMessages(library("tidyr"))
suppressMessages(library("tibble")) 
suppressMessages(library(future))
library("ggrepel")



[1] "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"

[1] "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"

# Create graph path

In [35]:
if(file.exists("/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/DE_graphs")){

        path_graphs<-"/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/DE_graphs"

}else{

    dir.create("/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/DE_graphs")
    path_graphs<-"/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/DE_graphs"
}

## Read in data

In [36]:
setwd("/group/soranzo/manuel.tardaguila/small_rna/snoRNAs/")

In [37]:
DE_results<-read.table(file="DE_results.tsv", sep="\t", header=T)

cat("DE_results\n")
str(DE_results)
cat("\n")

DE_results
'data.frame':	250 obs. of  9 variables:
 $ snoRNA        : chr  "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
 $ baseMean      : num  2.72 1.16 47.85 2.36 1.62 ...
 $ log2FoldChange: num  -4.82e-07 -3.19e-07 5.60e-06 7.28e-07 -2.14e-08 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.47762 0.51146 0.00501 0.26559 0.97 ...
 $ padj          : num  0.997 0.997 0.997 0.997 0.997 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...
 $ minuslog10padj: num  0.00118 0.00118 0.00118 0.00118 0.00118 ...
 $ abslogfc      : num  4.82e-07 3.19e-07 5.60e-06 7.28e-07 2.14e-08 ...



In [38]:
normalised_counts<-as.matrix(read.table(file="nor_counts.tsv", sep="\t", header=T))

cat("normalised_counts\n")
str(normalised_counts)
cat("\n")

normalised_counts
 num [1:250, 1:6] 0 0 25.1 1 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:250] "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...



In [39]:
SIG<-DE_results[which(DE_results$minuslog10padj >= 1.3),]

cat("SIG_0\n")
str(SIG)
cat("\n")

SIG_0
'data.frame':	0 obs. of  9 variables:
 $ snoRNA        : chr 
 $ baseMean      : num 
 $ log2FoldChange: num 
 $ lfcSE         : num 
 $ pvalue        : num 
 $ padj          : num 
 $ contrast      : chr 
 $ minuslog10padj: num 
 $ abslogfc      : num 



## Represent DE genes or if there are not the top 50 

In [40]:
DE_results<-DE_results[order(DE_results$minuslog10padj, decreasing = TRUE),]


cat("DE_results_1\n")
str(DE_results)
cat("\n")

selected<-DE_results[c(1:50),]

cat("selected_0\n")
str(selected)
cat("\n")

DE_results_1
'data.frame':	250 obs. of  9 variables:
 $ snoRNA        : chr  "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
 $ baseMean      : num  2.72 1.16 47.85 2.36 1.62 ...
 $ log2FoldChange: num  -4.82e-07 -3.19e-07 5.60e-06 7.28e-07 -2.14e-08 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.47762 0.51146 0.00501 0.26559 0.97 ...
 $ padj          : num  0.997 0.997 0.997 0.997 0.997 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...
 $ minuslog10padj: num  0.00118 0.00118 0.00118 0.00118 0.00118 ...
 $ abslogfc      : num  4.82e-07 3.19e-07 5.60e-06 7.28e-07 2.14e-08 ...

selected_0
'data.frame':	50 obs. of  9 variables:
 $ snoRNA        : chr  "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
 $ baseMean      : num  2.72 1.16 47.85 2.36 1.62 ...
 $ log2FoldChange: num  -4.82e-07 -3

In [41]:
DEBUG <- 1

In [42]:
 snoRNA_EXP_matrix<-normalised_counts[which(row.names(normalised_counts)%in%selected$snoRNA),]
       
       if(DEBUG == 1){
         
         cat("snoRNA_EXP_matrix_0\n")
         cat(str(snoRNA_EXP_matrix))
         cat("\n")
       }

snoRNA_EXP_matrix_0
 num [1:50, 1:6] 0 0 25.1 1 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:50] "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...



## annotation_col

In [43]:
 annotation_col<- data.frame(matrix(vector(), length(colnames(snoRNA_EXP_matrix)), 2,
                                          dimnames=list(c(),
                                                        c("Genotype","Batch"))),stringsAsFactors=F)
       
       row.names(annotation_col)<-colnames(snoRNA_EXP_matrix)
       

       
       if(DEBUG == 1){
         cat("annotation_col_0\n")
         cat(str(annotation_col))
         cat("\n")
         cat(str(row.names(annotation_col)))
         cat("\n")
         
       }
       
       
       annotation_col$Genotype<-gsub("_.+$","", row.names(annotation_col))
       annotation_col$Batch<-row.names(annotation_col)
    
       if(DEBUG == 1){
         cat("annotation_col_1\n")
         cat(str(annotation_col))
         cat("\n")
       }

 annotation_col$Genotype<-factor(annotation_col$Genotype,
                                       levels=c('wt','rs62237617'),
                                       ordered=T)

annotation_col$Batch<-factor(annotation_col$Batch,
                                       levels=c('wt_1','wt_2','wt_3',
                                               'rs62237617_1','rs62237617_2','rs62237617_3'),
                                       ordered=T)

      if(DEBUG == 1){
         cat("annotation_col_2\n")
         cat(str(annotation_col))
         cat("\n")
       }
       
      

annotation_col_0
'data.frame':	6 obs. of  2 variables:
 $ Genotype: logi  NA NA NA NA NA NA
 $ Batch   : logi  NA NA NA NA NA NA

 chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" "wt_3" ...

annotation_col_1
'data.frame':	6 obs. of  2 variables:
 $ Genotype: chr  "wt" "rs62237617" "wt" "rs62237617" ...
 $ Batch   : chr  "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...

annotation_col_2
'data.frame':	6 obs. of  2 variables:
 $ Genotype: Ord.factor w/ 2 levels "wt"<"rs62237617": 1 2 1 2 1 2
 $ Batch   : Ord.factor w/ 6 levels "wt_1"<"wt_2"<..: 1 4 2 5 3 6



## ann colors

In [44]:
vector_colors_Batch<- c(brewer.pal(9, "Greens")[c(5,7,9)],                                                 
                                                  brewer.pal(9, "Reds")[c(5,7,9)])

        names(vector_colors_Batch)<-levels(annotation_col$Batch)

vector_colors_Batch
        
        vector_colors_Genotype<-c(brewer.pal(9, "Greens")[c(5)],                                 
                                    brewer.pal(9, "Reds")[c(5)])
        
        names(vector_colors_Genotype)<-levels(annotation_col$Genotype)

vector_colors_Genotype

wt_1         wt_2         wt_3 rs62237617_1 rs62237617_2 rs62237617_3 
   "#74C476"    "#238B45"    "#00441B"    "#FB6A4A"    "#CB181D"    "#67000D"

wt rs62237617 
 "#74C476"  "#FB6A4A"

In [45]:
ann_colors <- list( Batch = vector_colors_Batch,
                           Genotype = vector_colors_Genotype)
      
       
       if(DEBUG == 1){
         cat("ann_colors_0\n")
         cat(str(ann_colors))
         cat("\n")
       }

ann_colors_0
List of 2
 $ Batch   : Named chr [1:6] "#74C476" "#238B45" "#00441B" "#FB6A4A" ...
  ..- attr(*, "names")= chr [1:6] "wt_1" "wt_2" "wt_3" "rs62237617_1" ...
 $ Genotype: Named chr [1:2] "#74C476" "#FB6A4A"
  ..- attr(*, "names")= chr [1:2] "wt" "rs62237617"



## Actual heatmap

In [46]:
heatmap<-pheatmap(snoRNA_EXP_matrix, display_numbers = FALSE,
                             show_colnames=FALSE,
                             angle_col = "0",
                             clustering_method="ward.D2",
                             fontsize_row = 8, 
                             fontsize_col = 8,
                             breaks=seq(-2,2,length.out=101),
                             color=colorRampPalette(c("blue","white","red"))(100),
                             scale="row",
                             cluster_cols=TRUE,
                             border_color='black',
                             treeheight_row=70, treeheight_col=70, cutree_cols=3,
                             annotation_col = annotation_col,
                             annotation_colors = ann_colors)



ERROR: Error in value[[3L]](cond): no active device and default getOption("device") is invalid


plot without title

In [ ]:
setwd(path_graphs)
         
svgname<-paste("Heatmap_snoRNAs",".svg",sep='')

ggsave(svgname,plot=heatmap, device ='svg')

# Volcano plot

In [47]:
cat("DE_results_1\n")
str(DE_results)
cat("\n")

DE_results_1
'data.frame':	250 obs. of  9 variables:
 $ snoRNA        : chr  "ENSG00000201882" "ENSG00000212626" "ENSG00000273885" "ENSG00000275146" ...
 $ baseMean      : num  2.72 1.16 47.85 2.36 1.62 ...
 $ log2FoldChange: num  -4.82e-07 -3.19e-07 5.60e-06 7.28e-07 -2.14e-08 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.47762 0.51146 0.00501 0.26559 0.97 ...
 $ padj          : num  0.997 0.997 0.997 0.997 0.997 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...
 $ minuslog10padj: num  0.00118 0.00118 0.00118 0.00118 0.00118 ...
 $ abslogfc      : num  4.82e-07 3.19e-07 5.60e-06 7.28e-07 2.14e-08 ...



## Actual volcano plot

In [48]:
 volcano_plot<-ggplot(data=DE_results,
                                aes(x=log2FoldChange,
                                    y=minuslog10padj)) +
             geom_vline(xintercept=c(-0.1,0.1), color="gray", linetype='dashed',linewidth=1)+
             geom_hline(yintercept=c(1.3), color="gray", linetype='dashed',linewidth=1)+
             geom_point(data=DE_results[which(DE_results$minuslog10padj < 1.3 & DE_results$abslogfc < 0.1),],
                        color="gray",fill="gray", stroke=0.2, shape=21, size=2)+
             geom_point(data=DE_results[which(DE_results$minuslog10padj >= 1.3 & DE_results$log2FoldChange <= -0.1),],
                        color="black",fill="blue", stroke=0.2, shape=21, size=4)+
             geom_point(data=DE_results[which(DE_results$minuslog10padj >= 1.3 & DE_results$log2FoldChange >= 0.1),],
                        color="black",fill="red", stroke=0.2, shape=21, size=4)+
             scale_y_continuous(name="-log10 pval")+
             theme_classic()+
             theme(plot.title=element_text(color="black", family="sans", size=14),
                   axis.title.y=element_text(color="black", family="sans", size=12),
                   axis.title.x=element_text(color="black", family="sans", size=12),
                   axis.text.y=element_text(color="black", family="sans", size=8),
                   axis.text.x=element_text(color="black", family="sans", size=8))+
             theme(legend.title = element_text(family="sans"),
                   legend.text = element_text(family="sans"),
                   legend.key.size = unit(0.35, 'cm'), #change legend key size
                   legend.key.height = unit(0.35, 'cm'), #change legend key height
                   legend.key.width = unit(0.35, 'cm'), #change legend key width
                   legend.position="hidden")+
             ggeasy::easy_center_title()
           
           volcano_plot <-volcano_plot+
             geom_text_repel(data=DE_results[which(DE_results$minuslog10padj >= 1.3 & DE_results$log2FoldChange <= -0.1),],
                             aes(x=log2FoldChange,
                                 y=minuslog10padj,
                                 label=snoRNA),
                             family="sans",
                             fontface='italic',
                             color='blue',
                             segment.size  = 0.25,
                             segment.color = "blue",
                             force=25,
                             size=4,
                             box.padding = 1,
                             max.overlaps = Inf)
           
           volcano_plot <-volcano_plot+
             geom_text_repel(data=DE_results[which(DE_results$minuslog10padj >= 1.3 & DE_results$log2FoldChange >= 0.1),],
                             aes(x=log2FoldChange,
                                 y=minuslog10padj,
                                 label=snoRNA),
                             family="sans",
                             fontface='italic',
                             color='red',
                             segment.size  = 0.25,
                             segment.color = "red",
                             force=25,
                             size=4,
                             box.padding = 1,
                             max.overlaps = Inf)

#              ggtitle(paste(paste(paste(contrast_sel, sep=''), paste(identity_sel, sep=''),sep='  '),paste(dim(SIG_genes)[1], "DE genes", sep=" "), sep="\n"))+


volcano_plot

ERROR: Error in value[[3L]](cond): no active device and default getOption("device") is invalid


plot without title

In [ ]:
setwd(path_graphs)
         
svgname<-paste("Volcano_snoRNAs",".svg",sep='')

ggsave(svgname,plot=volcano_plot, device ='svg')